In [2]:
#import required libraries for webscraping using selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import requests
from tabula.io import read_pdf
import pandas as pd

In [ ]:
# write a python code to extract link from following webpage https://www.pbs.gov.au/info/industry/listing/elements/pbac-meetings/pbac-outcomes (do not give previous suggestion)
url = "https://www.pbs.gov.au/info/industry/listing/elements/pbac-meetings/pbac-outcomes"
driver = webdriver.Chrome(executable_path=r"E:/Usecases/Webscrape/chromedriver.exe")
driver.get(url)
outcome_list =[] 
outcome_links = driver.find_elements_by_xpath('//*[@id="1.1251"]/ul[1]/li')
for link in outcome_links:
    outcome_list.append(link.find_elements_by_tag_name("a")[0].get_attribute("href"))
# driver.close()

In [ ]:
inner_links = []
for link in outcome_list:
    driver.get(link)
    # time.sleep(2)
    links = driver.find_elements_by_xpath('//*[@id="2"]/ul/li')
    inner_link_temp = []
    for inner_link in links:
        inner_link_temp.append(inner_link.find_elements_by_tag_name("a")[0].get_attribute("href"))
    inner_links.append(inner_link_temp)



In [ ]:
inner_links

In [ ]:
df_list = []
for item in outcome_list:
    if item.endswith(".pdf"):
        file_name = item.split("/")[-1]
        pdf_response = requests.get(item)
        with open("./Download/"+file_name, "wb") as f:
            f.write(pdf_response.content)
        print(f"Downloaded {file_name} from {item}")


In [ ]:
#iterate all files in the folder and extract table from each pdf file
for file in os.listdir("./Download"):
    if file.endswith(".pdf"):
        df_list.append(read_pdf("./Download/"+file, pages="all", multiple_tables=True, pandas_options={'header': None}))
        print(f"Extracted table from {file}")

In [ ]:
#write function to extract table from pdf file without using tabula
def extract_table(pdf_file):
    # Use tabula to read the PDF file and extract the table
    table = tabula.read_pdf(pdf_file, pages="all")
    # Return the table as a pandas dataframe
    return table
#write another function with diffrent approach using camelot
def extract_table_camelot(pdf_file):
    # Use tabula to read the PDF file and extract the table
    tables = camelot.read_pdf(pdf_file, pages="all")
    # Return the table as a pandas dataframe
    return tables


In [ ]:
# Write a function to anagram of a string
def anagram(string1, string2):
    # Write your code here
    if len(string1) != len(string2):
        return False
    else:
        string1 = string1.lower()
        string2 = string2.lower()
        for char in string1:
            if char in string2:
                string2 = string2.replace(char, "", 1)
            else:
                return False
        return True


In [ ]:
# Write python code to find seasonal index of a given data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA

# Read the data
df = pd.read_csv("https://raw.githubusercontent.com/rajeevratan84/datascienceforbusiness/master/monthly_milk_production.csv")
df.head()

# Convert the date column to datetime format
df['Month'] = pd.to_datetime(df['Month'])

# Set the date column as index
df.set_index('Month', inplace=True)

# Plot the data
df.plot(figsize=(12, 6))
plt.show()

# Decompose the data
decomposition = seasonal_decompose(df, model='additive')
decomposition.plot()
plt.show()

# Check for stationarity
result = adfuller(df['Production'])
print('ADF Statistic: {}'.format(result[0]))
print('p-value: {}'.format(result[1]))

# Plot the ACF and PACF plots
plot_acf(df, lags=20)
plot_pacf(df, lags=20)
plt.show()

# Split the data into train and test
train = df[:156]
test = df[156:]

# Build the model
model = ARIMA(train, order=(1, 1, 1))
model_fit = model.fit()

# Predict the values
predictions = model_fit.forecast(steps=12)[0]

# Plot the predictions
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

#calculate accuracy of the model
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(test, predictions))
print(rmse)




In [ ]:
#write a python function to calculate "seasonality index" by quarter
def seasonality_index(df):
    # Write your code here
    df['Month'] = pd.to_datetime(df['Month'])
    df.set_index('Month', inplace=True)
    decomposition = seasonal_decompose(df, model='additive')
    return decomposition.seasonal.groupby(decomposition.seasonal.index.quarter).mean()

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO

# Path to your PDF file
pdf_file = 'path/to/your/pdf/file.pdf'

# Path to the output XML file
xml_file = 'path/to/output/xml/file.xml'

# Open the PDF file in binary mode
with open(pdf_file, 'rb') as fp:
    # Create a PDF parser object
    parser = PDFParser(fp)
    # Create a PDF document object
    document = PDFDocument(parser)

    # Create a PDF resource manager object
    rsrcmgr = PDFResourceManager()

    # Create a BytesIO object to store the XML output
    xml_data = BytesIO()

    # Set up parameters for the XML converter
    laparams = LAParams()

    # Create a PDF to XML converter object
    converter = XMLConverter(rsrcmgr, xml_data, laparams=laparams)

    # Create a PDF interpreter object
    interpreter = PDFPageInterpreter(rsrcmgr, converter)

    # Iterate over each page in the PDF document
    for page in PDFPage.create_pages(document):
        # Process the page
        interpreter.process_page(page)

    # Get the XML data from the BytesIO object
    xml_data.seek(0)
    xml_content = xml_data.read().decode()

    # Write the XML content to the output file
    with open(xml_file, 'w', encoding='utf-8') as xml_fp:
        xml_fp.write(xml_content)


In [ ]:
import win32com.client
import os

def read_table(table):
  """Reads a table with vertically merged cells in Win32Com Word.

  Args:
    table: The table object to read.

  Returns:
    A list of lists, where each sublist represents a row in the table.
  """

  rows = []
  for row in table.Rows:
    row_data = []
    for cell in row.Cells:
      if cell.MergedCells.Count > 0:
        row_data.append("")
      else:
        row_data.append(cell.Range.Text)
    rows.append(row_data)
  return rows

# def main():
#   """Reads a table with vertically merged cells in Win32Com Word and prints it out."""

#   word = win32com.client.Dispatch("Word.Application")
#   doc = word.Documents.Open("my_document.docx")
#   table = doc.Tables(1)
#   rows = read_table(table)
#   for row in rows:
#     print(row)

# if __name__ == "__main__":
#   main()


In [ ]:
word = win32com.client.Dispatch("Word.Application")
doc = word.Documents.Open(os.path.abspath("./Download/pbac-web-outcomes-11-2021-v2.docx"))
table = doc.Tables(1)
# rows = read_table(table)
# for row in rows:
#     print(row)

# word = win32com.client.Dispatch("Word.Application")
# doc = word.Documents.Open("MyDocument.docx")
# table = doc.Tables("Table1")

# for row in table.Rows:
#     if row.MergeCells:
#         row.Unmerge()

# doc.Save()

# for row in range(table.Rows.Count):
#     for cell in table.Rows[row].Cells:
#         value = cell.Value
#         print(value)

# doc.Save()

for col in table.Columns:
    if col.IsMerged:
        col.Unmerge()
doc.Save()


In [ ]:
#resolve following issue "com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Word', 'Cannot access individual rows in this collection because the table has vertically merged cells.', 'wdmain11.chm', 25471, -2146822297), None)"
#https://stackoverflow.com/questions/64884102/how-to-read-table-with-vertically-merged-cells-in-win32com-word


In [ ]:
#write a python function to read a table with vertically merged cells in Win32Com Word
import win32com.client
import os

def read_table(table):
    """Reads a table with vertically merged cells in Win32Com Word.
    
    Args:
        table: The table object to read.
    
    Returns:
        A list of lists, where each sublist represents a row in the table.
    """
    
    rows = []
    for row in table.Rows:
        row_data = []
        for cell in row.Cells:
            if cell.MergedCells.Count > 0:
                row_data.append("")
            else:
                row_data.append(cell.Range.Text)
        rows.append(row_data)
    return rows

def main():
    """Reads a table with vertically merged cells in Win32Com Word and prints it out."""
    
    word = win32com.client.Dispatch("Word.Application")
    doc = word.Documents.Open("my_document.docx")
    table = doc.Tables(1)
    rows = read_table(table)
    for row in rows:
        print(row)

#above function is not working
#https://stackoverflow.com/questions/64884102/how-to-read-table-with-vertically-merged-cells-in-win32com-word

In [ ]:
#write a function to extract table from pdf using camelot
import camelot
tables = camelot.read_pdf('./Download/pbac-web-outcomes-11-2021-v2.pdf', flavor='stream')
tables.export('foo.csv', f='csv', compress=True) # json, excel, html, sqlite
tables[0].parsing_report


In [ ]:
import cv2

In [ ]:
import subprocess
import json
import os


def extract_tables_from_pdf(pdf_path):
    try:
        output = subprocess.check_output(['java', '-jar', 'pdfbox-app-2.0.29.jar', 'ExtractText', '-json', pdf_path], stderr=subprocess.STDOUT)
        json_data = output.decode('utf-8')
        extracted_tables = []

        try:
            data = json.loads(json_data)
            pages = data['pages']

            for page in pages:
                tables = page['tables']
                for table in tables:
                    extracted_table = []
                    rows = table['rows']
                    for row in rows:
                        cells = row['cells']
                        extracted_row = [cell['text'] for cell in cells]
                        extracted_table.append(extracted_row)
                    extracted_tables.append(extracted_table)

        except json.JSONDecodeError:
            print('Error decoding JSON data.')

        return extracted_tables

    except subprocess.CalledProcessError as e:
        print('Error running command: ', e.output.decode('utf-8'))
        return None


pdf_path = os.path.abspath('./Download/pbac-web-outcomes-11-2021-v2.pdf')
tables = extract_tables_from_pdf(r"E:\Usecases\Webscrape\Download\pbac-web-outcomes-11-2021-v2.pdf")
print(tables)


In [7]:
import subprocess
import json


def extract_tables_from_pdf(pdf_path):
    command = ['java', '-jar', 'pdfbox-app-2.0.26.jar', 'ExtractText', '-json', pdf_path]
    process = subprocess.run(command, capture_output=True, text=True)
    json_data = process.stdout
    print(json_data, type(json_data))
    extracted_tables = []

    try:
        data = json.loads(json_data)
        pages = data['pages']

        for page in pages:
            tables = page['tables']
            for table in tables:
                extracted_table = []
                rows = table['rows']
                for row in rows:
                    cells = row['cells']
                    extracted_row = [cell['text'] for cell in cells]
                    extracted_table.append(extracted_row)
                extracted_tables.append(extracted_table)

    except json.JSONDecodeError:
        print('Error decoding JSON data.')

    return extracted_tables

pdf_path = os.path.abspath('./Download/pbac-web-outcomes-11-2021-v2.pdf')
tables = extract_tables_from_pdf("./Download/pbac-web-outcomes-11-2021-v2.pdf")
print(tables)

 <class 'str'>
Error decoding JSON data.
[]


In [8]:
#Write a python function to extract table from pdf using python-docx


[{'DRUG NAME, FORM(S), STRENGTH(S), SPONSOR, TYPE OF SUBMISSION': 'ABROCITINIB \n\nTablet 100 mg\nTablet 200 mg\n\nCibinqo®\n\nPfizer Australia Pty Ltd\n\nCategory 2 submission\n(New PBS listing) ', 'DRUG TYPE AND USE\n': 'Atopic dermatitis', 'LISTING REQUESTED BY SPONSOR / PURPOSE OF SUBMISSION': 'To request a General Schedule Authority Required listing for the treatment of severe atopic dermatitis.', 'PBAC OUTCOME\n': 'To be considered at a future PBAC meeting'}, {'DRUG NAME, FORM(S), STRENGTH(S), SPONSOR, TYPE OF SUBMISSION': 'ACALABRUTINIB \n\nCapsule 100 mg\n\nCalquence®\n\nAstraZeneca Pty Ltd\n\nStandard Re-entry submission\n(Change to PBS listing) ', 'DRUG TYPE AND USE\n': 'Chronic lymphocytic leukaemia (CLL) or small lymphocytic lymphoma (SLL)', 'LISTING REQUESTED BY SPONSOR / PURPOSE OF SUBMISSION': 'Resubmission to request a General Schedule Authority Required listing for previously untreated CLL or SLL patients who are considered unsuitable for treatment with fludarabine-bas

In [9]:
import docx
import pandas as pd

def extract_table(pdf_file):
  # Create a Word document object
  doc = docx.Document(pdf_file)
  # Create an empty list to store the tables
  tables = []
  # Loop through the tables in the document
  for table in doc.tables:
    # Create an empty list to store the rows
    rows = []
    # Loop through the rows in the table
    for row in table.rows:
      # Create an empty list to store the cells
      cells = []
      # Loop through the cells in the row
      for cell in row.cells:
        # Add the text of the cell to the list
        cells.append(cell.text)
      # Add the list of cells to the list of rows
      rows.append(cells)
    # Add the list of rows to the list of tables
    tables.append(rows)
  # Return the list of tables as a pandas dataframe
  return pd.DataFrame(tables[0][1:], columns=tables[0][0])

In [10]:
extract_table("./Download/pbac-web-outcomes-11-2021-v2.pdf")

PackageNotFoundError: Package not found at './Download/pbac-web-outcomes-11-2021-v2.pdf'

In [1]:
from docx import Document


def extract_tables_from_docx(docx_path):
    doc = Document(docx_path)
    extracted_tables = []

    for table in doc.tables:
        extracted_table = []
        for row in table.rows:
            extracted_row = [cell.text for cell in row.cells]
            extracted_table.append(extracted_row)
        extracted_tables.append(extracted_table)

    return extracted_tables

docx_path = './Download/2021-11-05-PBAC-Web-Outcomes.docx'
tables = extract_tables_from_docx(docx_path)
print(tables)


PackageNotFoundError: Package not found at './Download/2021-11-05-PBAC-Web-Outcomes.docx'

In [11]:
import docx
def read_table_from_word_doc(doc_path): 
    doc = docx.Document(doc_path)
    tables = doc.tables
    tbl_list = []
    for table in tables:
        if len(table.columns) < 4:
            continue
        data = []
        keys = None
        for i, row in enumerate(table.rows):
            text = (cell.text for cell in row.cells)
            if i == 0:
                keys = tuple(text)
                continue
            row_data = dict(zip(keys, text))
            data.append(row_data)
    tbl_list.append(pd.DataFrame(data))
    return pd.concat(tbl_list).reset_index(drop=True)

In [12]:
import os
doc_path = os.path.abspath('./Download/2021-11-05-PBAC-Web-Outcomes.docx')
read_table_from_word_doc(doc_path)


PackageNotFoundError: Package not found at 'e:\Usecases\Webscrape\Download\2021-11-05-PBAC-Web-Outcomes.docx'